In [1]:
%load_ext autoreload
%autoreload 2
import sys
# do this to be able to import the custom python scripts
sys.path.insert(1, "../../../python_scripts")
import os

from recordlinkage import preprocessing

import json
import pandas as pd
import csv
import re
import numpy as np
import datetime

import dm_file_checker

## Considerations in Data Cleaning

1. Incorrect CSV formatting (if data is in CSV format)
2. Duplicate primary key
3. Thorough inspection of null encodings
    - null value, "None", "nan", "", "missing", "unknown", "UNK", -1, 0, "not applicable", "NA", "none", "not available"
    - all of these different encodings should be encoded to be the same null value
    - for government ID numbers (e.g. SSS, TIN), check their value counts (i.e. count per unique value). Counts should be as close to 1 as possible. A value with count in the hundreds could signal a null encoding (i.e. 123456789 as a default ID number)
4. Removal of columns with too many missing entries
5. Replacing common abbreviations
    - Example: Ma. vs Maria, Sta. vs Santa
6. Inspecting non-numeric values in numeric fields
7. Inspecting anomaly values in numeric fields
    - Examples: 1900 for birth year, negative values for price
8. Inspecting records with missing names (because these are dubious accounts)
9. Inspecting records with exact match in all fields
10. Ensuring strings are all in ASCII encoding
11. Ensure dates are encoded directly
    - Example: no Feb 30
12. More in-depth cleaning (in order)
    1. Converting all alphabetical characters to lowercase
    2. Strip accents
        - Accented characters should be converted to their corresponding ASCII
        - Example:
            - The ñ character being sometimes encoded as n. 
            - The ñ character should be properly read in memory beforehand (utf-8 encoding, not ASCII)
    3. Removing all content between brackets and the brackets itself. Brackets are either [], (), or {} 
    4. Replace all special characters with the empty string "" 
        - NON-special charactes by default are the following: hyphen "-", the underline "_", the comma, ",", the alphabet, and numbers
    5. Replace with whitespace " "
        - The following by default are replaced with whitespaces: hyphen "-", underline "_", comma ","
    6. Remove consecutive whitespaces
    7. Remove trailing and leading whitespaces
13. After all these steps, ensure remaining characters are 0 to 9, a to z (all lowercase), and whitespace, for a total of 37 characters
    
## Considerations in Data Cleaning for Record Linkage Specifically

1. Consistency of encodings 
    - Example 1: one table uses F and M for gender but another table uses 1 and 0
    - Example 2: Feb vs 02 vs February
    - Standardizing date conventions 
        - Example: DD/MM/YYYY vs YYYYMMDD vs MM/DD/YYYY
        
2. Consistency of fields across tables. 
    - Example: first name, last name as separate fields in Table A but full name as one string in Table B. In this case, recommended to segment strings in Table B rather than concatenate strings in Table A so there is a surname to surname comparison and a first name to first name comparison.
    
3. Uniqueness of primary key across tables
    - There shouldn't be a key that's the same across tables (i.e. ID 1001 in table A and ID 1001 in table B)

## Get Appropriate Filepaths

In [2]:
saved_files_path = "../../../saved_files"
task_name = os.path.basename(os.getcwd())
dataset_name = task_name.split("-")[1]

# files to read in
raw_data_filepath = dm_file_checker.get_filepath(task_name, "raw_data", saved_files_path)

primary_key = dm_file_checker.get_dataset_info(task_name, "primary_key", saved_files_path)
numeric_fields = dm_file_checker.get_dataset_info(task_name, "numeric_fields", saved_files_path)
date_fields = dm_file_checker.get_dataset_info(task_name, "date_fields", saved_files_path)
    
# files to write out
unlabeled_data_filepath = dm_file_checker.get_filepath(task_name, "unlabeled_data", saved_files_path)
cleaned_strings_folder =  dm_file_checker.get_filepath(task_name, "cleaned_strings_folder", saved_files_path)
value_counts_folder = dm_file_checker.get_filepath(task_name, "value_counts_folder", saved_files_path)

# Set whether or not to write out files for manual inspection
- Set the two boolean variable below. For big files, you may want to set this to True in a initial run, then False for subsequent runs.
- Value frequency counts per field
- Before and after cleaning per field

In [3]:
WRITE_OUT_VALUE_COUNTS = True
WRITE_OUT_CLEANED_STRINGS = True

# Set what the agreed upon encoding for null
- Set as the empty string

In [4]:
CANON_NULL_ENCODING = ""

## Read in Raw Data

In [5]:
raw_data = []

with open(raw_data_filepath, "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            header = row
        else:
            raw_data.append(row)
        line_count += 1

## Check How Much of CSV Was Delimited Correctly

In [6]:
bad_data = [row for row in raw_data if len(row) != len(header)]

print("Number of incorrectly delimited rows: {:,}".format(len(bad_data)))
print("Percentage of rows incorrectly delimited: {:.1f}%".format(len(bad_data)*100/len(raw_data)))

Number of incorrectly delimited rows: 0
Percentage of rows incorrectly delimited: 0.0%


In [7]:
if len(bad_data) != 0:
    raw_data = [row for row in raw_data if len(row) == len(header)]
print("Number of correctly delimited rows: {:,}".format(len(raw_data)))

del bad_data
raw_data = pd.DataFrame(raw_data, columns = header)
raw_data.head()

Number of correctly delimited rows: 5,000


,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
0,rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974
1,rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216
2,rec-988-dup-1,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128,2185997
3,rec-1716-dup-1,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,19921119,4314184
4,rec-1213-org,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,19991207,9144092


## Check Duplicate Primary Keys

In [8]:
num_dup_primary_key = raw_data[primary_key].duplicated().sum()
print("Number of rows with duplicate primary key: {}".format(num_dup_primary_key))

Number of rows with duplicate primary key: 0


## Removal of Exact Duplicates

In [9]:
duplicated_rows = raw_data.duplicated(keep = "first")
print("Removing {} rows that are exact duplicates".format(duplicated_rows.sum()))
raw_data = raw_data.loc[~duplicated_rows,:]

Removing 0 rows that are exact duplicates


## Keeping only first occurence of duplicate primary key (if applicable)

In [10]:
# non_first_duplicate_primary_key = (~raw_data[primary_key].duplicated(keep = "first")) \
#                                     & raw_data[primary_key].duplicated(keep = False)
# print("Removing {} rows to keep only the first occurence of duplicate primary key".format(non_first_duplicate_primary_key.sum()))
# raw_data = raw_data.loc[~non_first_duplicate_primary_key,:]

## Making a New Primary Key (if applicable)
- new primary key is based on DataFrame index

In [11]:
# raw_data = raw_data.rename(columns = {primary_key:"{}_old".format(primary_key)})
# raw_data.index.name = primary_key
# raw_data = raw_data.reset_index()
# raw_data[primary_key] = raw_data[primary_key] + 1

# raw_data.head()

In [12]:
# # write out mapping from old primary key to new primary key
# primary_key_mapping = raw_data.loc[:,[primary_key, "{}_old".format(primary_key)]]
# primary_key_mapping_filepath = raw_data_filepath.replace("raw_data.csv", "primary_key_mapping.csv")
# primary_key_mapping.to_csv(primary_key_mapping_filepath, index = False, quoting = csv.QUOTE_ALL)
# del primary_key_mapping

# raw_data = raw_data.drop(columns = "{}_old".format(primary_key))
# raw_data.head()

## Prepending Dataset Name to Primary Key
- For uniqueness of primary key across datasets

In [13]:
raw_data[primary_key] = raw_data[primary_key].apply(lambda x: "{}-{}".format(dataset_name, x))

raw_data = raw_data.set_index(primary_key)
raw_data.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974
febrl3-rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216
febrl3-rec-988-dup-1,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128,2185997
febrl3-rec-1716-dup-1,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,19921119,4314184
febrl3-rec-1213-org,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,19991207,9144092


## Remove Dummy Records (if applicable)
- Dummy records are records inserted as a workaround but don't actually represent an actual person or entity

In [14]:
# dummy customer removal code here

## Remove columns irrelevant for deduplication and record linkage
- Example: source table column, because it doesn't actually identify the entity or person

# FILL THE LIST BELOW WITH IRRELEVANT COLUMN NAMES

In [15]:
irrelevant_columns = []

In [16]:
raw_data = raw_data.drop(columns = irrelevant_columns)
raw_data.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974
febrl3-rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216
febrl3-rec-988-dup-1,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128,2185997
febrl3-rec-1716-dup-1,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,19921119,4314184
febrl3-rec-1213-org,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,19991207,9144092


# Convert Age to Birthdate (if applicable)

In [17]:
# CUTOFF_DATE = datetime.datetime(year = 2019, month = 9, day = 30)

# def convert_ageindays_to_birthdate(age, cutoff_date = CUTOFF_DATE):
#     try:
#         birthdate = CUTOFF_DATE - datetime.timedelta(days = age)
#         birthdate = birthdate.strftime("%Y/%m/%d")
#     except OverflowError:
#         birthdate = ""
    
#     return birthdate

In [18]:
# age_field = None
# if age_field is not None:
#     raw_data["birth_date"] = raw_data[age_field].apply(convert_ageindays_to_birthdate)
#     raw_data = raw_data.drop(columns = age_field)

## Convert All Strings to Lower Case (if applicable)
- enummerate all fields to convert to lower case below in `string_fields` list

In [19]:
# for field in raw_data.columns:
#     raw_data[field] = raw_data[field].str.lower()
# raw_data.head()

## Check for Frequency Value Counts
- To see which values occur the most often
- Useful for checking different encoding of nulls
- Also useful for checking common abbreviations

In [20]:
for col in raw_data.columns:
    value_counts_df = pd.DataFrame(raw_data[col].value_counts())
    value_counts_df.columns = ["count"]
    value_counts_df.index.name = col
    
    if WRITE_OUT_VALUE_COUNTS:
        filepath = os.path.join(value_counts_folder, col + ".csv")
        if not os.path.exists(value_counts_folder):
            os.mkdir(value_counts_folder)
            
        value_counts_df.to_csv(filepath)
    
    # check the top values
    print(value_counts_df.head(15))
    print("")

            count
given_name       
              156
joshua         81
emiily         69
jack           61
benjamin       54
isabella       51
samuel         49
thomas         47
sophie         46
james          45
william        42
alexandra      39
jessica        39
jacob          39
oliver         37

          count
surname        
white       123
clarke       86
             79
campbell     73
ryan         69
green        57
reid         50
dixon        47
nguyen       46
matthews     44
morrison     44
webb         44
mason        40
mccarthy     39
shepherd     38

               count
street_number       
                 245
1                166
3                159
5                150
8                134
4                132
16               119
7                117
11               111
12               110
6                107
10               106
2                102
19               101
15                96

                       count
address_1                   
    

## Check for Null Encodings
- null value, "None", "", "nan", "missing", "unknown", "UNK", -1, 0, "not applicable", "NA", "none", "not available"
- all of these different encodings should be encoded to be the same null value
- for government ID numbers (e.g. SSS, TIN), check their value counts (i.e. count per unique value). Counts should be as close to 1 as possible. A value with count in the hundreds could signal a null encoding (i.e. 123456789 as a default ID number)


In [21]:
# checking if there are native Python nulls (these are np.nan and None)
raw_data.isnull().sum()

given_name       0
surname          0
street_number    0
address_1        0
address_2        0
suburb           0
postcode         0
state            0
date_of_birth    0
soc_sec_id       0
dtype: int64

## Automatically Setting Value Count Threshold for Suspected Null Encodings (if applicable)
- E.g. a social security number occuring more than 100 times is likely to be a null encoding
- Other examples: Mobile Number, Landline Number, Email, Government ID Numbers

In [22]:
threshold_null_encodings = []
unique_identifier_fields = ["soc_sec_id"]

VALUE_COUNT_THRESHOLD = 100

In [23]:
for field in unique_identifier_fields:
    value_counts_df = raw_data[field].value_counts().sort_values(ascending = False)
    addtl_null_encodings = value_counts_df[(value_counts_df > VALUE_COUNT_THRESHOLD)].index.tolist()
    threshold_null_encodings += addtl_null_encodings
    print("Adding the following null encodings from {} that occur at least {} times: {}".format(field, VALUE_COUNT_THRESHOLD, addtl_null_encodings))

threshold_null_encodings = list(set(threshold_null_encodings))

Adding the following null encodings from soc_sec_id that occur at least 100 times: []


# FILL THE LIST BELOW WITH SUSPECTED NULL ENCODINGS
- null encodings other than the canon null encodings
- Examples: "n/a", "not included", "not indicated", "00", "01/01/1900", "11111111" for ID Number

In [24]:
suspected_null_encodings = ["missing", "unknown", "not available"]

In [25]:
# check how many values per column per null encoding
for null_encoding in [CANON_NULL_ENCODING] + suspected_null_encodings:
    print("Checking for null encoding '{}'".format(null_encoding))
    total_null_encodings = (raw_data == null_encoding).sum(axis = 0)
    percentage_null_encodings = total_null_encodings*100/raw_data.shape[0]
    
    print(total_null_encodings)
    print("")
    print(percentage_null_encodings)
    print("")

Checking for null encoding ''
given_name       156
surname           79
street_number    245
address_1        154
address_2        693
suburb            85
postcode           0
state             85
date_of_birth    155
soc_sec_id         0
dtype: int64

given_name        3.12
surname           1.58
street_number     4.90
address_1         3.08
address_2        13.86
suburb            1.70
postcode          0.00
state             1.70
date_of_birth     3.10
soc_sec_id        0.00
dtype: float64

Checking for null encoding 'missing'
given_name       0
surname          0
street_number    0
address_1        0
address_2        0
suburb           0
postcode         0
state            0
date_of_birth    0
soc_sec_id       0
dtype: int64

given_name       0.0
surname          0.0
street_number    0.0
address_1        0.0
address_2        0.0
suburb           0.0
postcode         0.0
state            0.0
date_of_birth    0.0
soc_sec_id       0.0
dtype: float64

Checking for null encoding 'unkno

## Convert all null encodings to empty string

In [26]:
suspected_null_encodings = suspected_null_encodings + threshold_null_encodings
print("Will convert all these null encodings: {}".format(suspected_null_encodings))

Will convert all these null encodings: ['missing', 'unknown', 'not available']


In [27]:
# converting native Python nulls to canon null encoding
raw_data = raw_data.fillna(value = CANON_NULL_ENCODING)

# converting all other null encodings to the canon null encoding
for null_encoding in suspected_null_encodings:
    raw_data = raw_data.replace(to_replace = null_encoding, value = CANON_NULL_ENCODING)
raw_data.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974
febrl3-rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216
febrl3-rec-988-dup-1,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128,2185997
febrl3-rec-1716-dup-1,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,19921119,4314184
febrl3-rec-1213-org,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,19991207,9144092


## Remove Columns with Too Many Nulls

In [28]:
# final check of null encodings
print("Checking for null encoding '{}'".format(CANON_NULL_ENCODING))
total_null_encodings = (raw_data == CANON_NULL_ENCODING).sum(axis = 0)
percentage_null_encodings = total_null_encodings*100/raw_data.shape[0]

print(total_null_encodings)
print("")
print(percentage_null_encodings)
print("")

Checking for null encoding ''
given_name       156
surname           79
street_number    245
address_1        154
address_2        693
suburb            85
postcode           0
state             85
date_of_birth    155
soc_sec_id         0
dtype: int64

given_name        3.12
surname           1.58
street_number     4.90
address_1         3.08
address_2        13.86
suburb            1.70
postcode          0.00
state             1.70
date_of_birth     3.10
soc_sec_id        0.00
dtype: float64



# FILL THE LIST BELOW WITH COLUMNS TO REMOVE DUE TO NULLS

In [29]:
null_columns = []

In [30]:
raw_data = raw_data.drop(columns = null_columns)
raw_data.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974
febrl3-rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216
febrl3-rec-988-dup-1,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128,2185997
febrl3-rec-1716-dup-1,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,19921119,4314184
febrl3-rec-1213-org,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,19991207,9144092


## Inspect Non-Numeric Values in Numeric fields

- Note: Data type of these fields should still be strings. We're just checking if they CAN be converted to float, because they will be in the model later on.
- Examples: 
[useful link for python checking numeric](https://stackoverflow.com/questions/736043/checking-if-a-string-can-be-converted-to-float-in-python)

In [31]:
def check_cannot_float(x):
    try:
        float(x)
        not_float = False
    except ValueError:
        not_float = True
    return not_float

In [32]:
for field in numeric_fields:
    print("Checking non-numeric values for numeric field {}".format(field))
    cannot_float_field = "{}_cannot_float".format(field)
    raw_data[cannot_float_field] = raw_data[field].apply(check_cannot_float)
    print("Total non-numeric values are {}".format(raw_data[cannot_float_field].sum()))
    print("Percentage of non-numeric values is {:.1f}%".format(raw_data[cannot_float_field].mean()*100))
    
    print("Examples of non-numeric are the following")
    print(raw_data.loc[raw_data[cannot_float_field] == True,field].unique())
    
raw_data.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974
febrl3-rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216
febrl3-rec-988-dup-1,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128,2185997
febrl3-rec-1716-dup-1,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,19921119,4314184
febrl3-rec-1213-org,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,19991207,9144092


# Address what to do with non-numeric values of numeric fields here

In [33]:
# convert all of them to the canon null encoding
for field in numeric_fields:
    cannot_float_field = "{}_cannot_float".format(field)
    
    raw_data.loc[raw_data[cannot_float_field] == True,field] = CANON_NULL_ENCODING
    raw_data = raw_data.drop(columns = cannot_float_field)
    
raw_data.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974
febrl3-rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216
febrl3-rec-988-dup-1,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128,2185997
febrl3-rec-1716-dup-1,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,19921119,4314184
febrl3-rec-1213-org,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,19991207,9144092


## Check Anomaly Values in Numeric Fields
 - Examples: 1900 for birth year, negative values for age or price

## Anomaly Check: Negative Values

In [34]:
def negative_checker(x, canon_null_encoding = CANON_NULL_ENCODING):
    # assuming that non-numeric values are already converted to the empty string
    if x == canon_null_encoding:
        is_negative = False
    else:
        x = float(x)
        if x < 0:
            is_negative = True
        else:
            is_negative = False
    return is_negative

In [35]:
for field in numeric_fields:
    print("Checking negative values for numeric field {}".format(field))
    
    is_negative_field = "{}_is_negative".format(field)
    raw_data[is_negative_field] = raw_data[field].apply(negative_checker)
    print("Total negative values are {}".format(raw_data[is_negative_field].sum()))
    print("Percentage of negative values is {:.1f}%".format(raw_data[is_negative_field].mean()*100))
    
    print("Examples of negative values are the following")
    print(raw_data.loc[raw_data[is_negative_field] == True,field].unique())

# Address what to do with negative values of numeric fields here

In [36]:
# convert all of them to the canon null encoding
for field in numeric_fields:
    is_negative_field = "{}_is_negative".format(field)
    
    raw_data.loc[raw_data[is_negative_field] == True,field] = CANON_NULL_ENCODING
    raw_data = raw_data.drop(columns = is_negative_field)
    
raw_data.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974
febrl3-rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216
febrl3-rec-988-dup-1,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128,2185997
febrl3-rec-1716-dup-1,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,19921119,4314184
febrl3-rec-1213-org,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,19991207,9144092


## In Depth Cleaning of String Fields
- The previous data cleaning steps were more about data types and formatting (null values, non-numeric in numeric fields)
- The succeeding steps will be more about the contents itself

- More in-depth cleaning (in order)
    1. Converting all alphabetical characters to lowercase
    2. Strip accents
       - Accented characters should be converted to their corresponding ASCII
       - Example:
           - The ñ character being sometimes encoded as n. 
           - The ñ character should be properly read in memory beforehand (utf-8 encoding, not ASCII)
    3. Removing all content between brackets and the brackets itself. Brackets are either [], (), or {} 
    4. Replace all special characters with the empty string "" 
        - NON-special charactes by default are the following: hyphen "-", the underline "_", the comma ",", the alphabet, and numbers
    5. Replace with whitespace " "
        - The following by default are replaced with whitespaces: hyphen "-", underline "_", comma ","
    6. Remove consecutive whitespaces
    7. Remove trailing and leading whitespaces

# Write function/s for cleaning string fields here

In [37]:
def generic_clean(series):
    cleaned_series = preprocessing.clean(series, 
                                         lowercase = True,
                                         replace_by_none = '[^ ,\\-\\_A-Za-z0-9]+',
                                         replace_by_whitespace = '[,\\-\\_]',
                                         strip_accents = "ascii",
                                         remove_brackets = True,
                                         encoding = "utf-8")
    return cleaned_series

# abbreviations in Filipino names
def name_abbv_clean(name):
    name = re.sub(r"\bma\b", "maria", name)
    name = re.sub(r"\bsta\b", "santa", name)
    name = re.sub(r"\bjr\b", "junior", name)
    name = re.sub(r"\bsr\b", "senior", name)
    name = re.sub(r"\bsto\b", "santo", name)
    
    return name

def email_clean(series):
    cleaned_series = preprocessing.clean(series, 
                                     lowercase = True,
                                     replace_by_none = '[^ @,\\-\\_A-Za-z0-9]+',
                                     replace_by_whitespace = '[@,\\-\\_]',
                                     strip_accents = "ascii",
                                     remove_brackets = True,
                                     encoding = "utf-8")
    return cleaned_series

In [38]:
#name_fields = ["given_name", "surname", "address_1", "address_2", "suburb"]
name_fields = []

email_fields = []

string_fields = [field for field in raw_data.columns if (field not in numeric_fields) and (field not in date_fields)]
print(string_fields)

['given_name', 'surname', 'street_number', 'address_1', 'address_2', 'suburb', 'postcode', 'state', 'soc_sec_id']


In [39]:
for field in string_fields:
    if field not in email_fields:
        print("Cleaning field {}".format(field))
        cleaned_field = "{}_clean".format(field)
        raw_data[cleaned_field] = generic_clean(raw_data[field])

        if field in name_fields:
            raw_data[cleaned_field] = raw_data[cleaned_field].apply(name_abbv_clean)

        clean_vs_old_data = raw_data.loc[raw_data[cleaned_field] != raw_data[field],[cleaned_field,field]]\
                                      .drop_duplicates()

        if WRITE_OUT_CLEANED_STRINGS:
            filepath = os.path.join(cleaned_strings_folder,"{}.csv".format(field))
            if not os.path.exists(cleaned_strings_folder):
                os.mkdir(cleaned_strings_folder)
                
            clean_vs_old_data.to_csv(filepath)

        print("Changed the following strings (cleaned vs original version)")
        print(clean_vs_old_data.values)
        print("")       

Cleaning field given_name
Changed the following strings (cleaned vs original version)
[['alexa rose' 'alexa-rose']
 ['lachlq njohn' 'lachlq-njohn']
 ['jean claude' 'jean-claude']
 ['lachlan john' 'lachlan-john']
 ['papp horvath' 'papp-horvath']
 ['alexa ose' 'alexa-ose']
 ['john paul' 'john-paul']
 ['alexa rxose' 'alexa-rxose']
 ['jean lcaude' 'jean-lcaude']
 ['tommi lee' 'tommi-lee']
 ['taylor saige' 'taylor-saige']
 ['alex arose' 'alex-arose']
 ['darcy' "d'arcy"]]

Cleaning field surname
Changed the following strings (cleaned vs original version)
[['lillie hinrichs' 'lillie-hinrichs']
 ['evans zijlstra' 'evans-zijlstra']
 ['pettman south' 'pettman-south']
 ['jean claude' 'jean-claude']
 ['auch schwelk' 'auch-schwelk']
 ['brummer archer' 'brummer-archer']
 ['hampton smith' 'hampton-smith']
 ['papp horvath' 'papp-horvath']
 ['naar cafentas' 'naar-cafentas']
 ['watkins yall' 'watkins-yall']
 ['oday' "o'day"]
 ['james palmer' 'james-palmer']
 ['alexa rose' 'alexa-rose']
 ['lambden stewar

In [40]:
for field in email_fields:
    print("Cleaning field {}".format(field))
    cleaned_field = "{}_clean".format(field)
    raw_data[cleaned_field] = email_clean(raw_data[field])
    
    clean_vs_old_data = raw_data.loc[raw_data[cleaned_field] != raw_data[field],[cleaned_field,field]]\
                                  .drop_duplicates()

    if WRITE_OUT_CLEANED_STRINGS:
        filepath = os.path.join(cleaned_strings_folder,"{}.csv".format(field))
        clean_vs_old_data.to_csv(filepath)

    print("Changed the following strings (cleaned vs original version)")
    print(clean_vs_old_data.values)
    print("")       

In [41]:
# dropping the old columns and renaming the clean version
raw_data = raw_data.drop(columns = string_fields)
for field in string_fields:
    raw_data = raw_data.rename(columns = {"{}_clean".format(field):field})
raw_data.head()

,date_of_birth,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,19560409,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,1804974
febrl3-rec-552-dup-3,19080419,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,6089216
febrl3-rec-988-dup-1,19081128,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,2185997
febrl3-rec-1716-dup-1,19921119,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,4314184
febrl3-rec-1213-org,19991207,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,9144092


## In-Depth Cleaning of Date Related Fields
- Segmenting
    - Segmenting the date field into day, month year
    - Why segment? Because more information will be used when linking and deduplication later on

- Cleaning
    - Making sure months fall between 1 to 12
    - Making sure the day count aligns with the number of days per month (adjusting for February leap years)

# MODIFY FUNCTIONS BELOW DEPENDING ON HOW DATE FIELD IS STRUCTURED

In [42]:
def is_valid_date(year, month, day, year_threshold = 1900):
    day_count_for_month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
    if year%4==0 and (year%100 != 0 or year%400==0):
        day_count_for_month[2] = 29
    
    is_month_valid = (1 <= month <= 12)
    
    is_year_valid = (year >= year_threshold)
    
    if is_month_valid:
        is_day_valid = (1 <= day <= day_count_for_month[month])
    else:
        is_day_valid = (1 <= day <= 31)
        
    return is_year_valid, is_month_valid, is_day_valid

def check_invalid_date(date, canon_null_encoding = CANON_NULL_ENCODING):
    if date == canon_null_encoding:
        is_date_invalid = True
    else:
        date = str(date)
        
        ## insert how to parse date string here
        #day, month, year = date.split("/")
        
        year = date[:4]
        month = date[4:6]
        day = date[6:8]
        
        try:
            year = int(year)
        except ValueError:
            year = -1
        try:
            month = int(month)
        except ValueError:
            month = -1
        try:
            day = int(day)
        except ValueError:
            day = -1
            
        is_year_valid, is_month_valid, is_day_valid = is_valid_date(year, month, day)
        
        is_date_valid = is_year_valid and is_month_valid and is_day_valid
        is_date_invalid = not is_date_valid
    
    return is_date_invalid

def standardize_date(date, canon_null_encoding = CANON_NULL_ENCODING):
    if date == canon_null_encoding:
        standardized_date = canon_null_encoding
    else:
        date = str(date)
        
        ## insert how to parse string here
        #day, month, year = date.split("/")
        
        year = date[:4]
        month = date[4:6]
        day = date[6:8]
        
        standardized_date = "{}/{}/{}".format(year, month, day)
        
    return standardized_date

## Anomaly Check: Invalid Date Fields

In [43]:
for field in date_fields:
    print("Checking invalid dates for {}".format(field))
    is_invalid_date = "{}_is_invalid".format(field)
    
    raw_data[is_invalid_date] = raw_data[field].apply(lambda date: check_invalid_date(date))
    print("Number of invalid {} values are {}".format(field, raw_data[is_invalid_date].sum()))
    print("Percent of invalid {} values is {:.1f}%".format(field, raw_data[is_invalid_date].mean()*100))

    print("Examples of invalid dates are the following")
    print(raw_data.loc[raw_data[is_invalid_date] == True,field].unique())

Checking invalid dates for date_of_birth
Number of invalid date_of_birth values are 190
Percent of invalid date_of_birth values is 3.8%
Examples of invalid dates are the following
['19551192' '' '19404502' '19390184' '19860896' '19160017' '19372109'
 '19741923' '19269828' '19448720' '19875031' '19720015' '19219503'
 '19600545' '19501711' '19851618' '19221620' '19505806' '19813005'
 '19400029' '19738013' '19475027' '19806817' '19320239' '19320079'
 '19790281' '19027803' '19545129' '19261198' '19310175' '19590398'
 '19281501' '19670273' '19903925' '19504306' '19214019']


# Address what to do with invalid dates here

In [44]:
# convert all of them to the canon null encoding
for field in date_fields:
    is_invalid_date = "{}_is_invalid".format(field)
    
    raw_data.loc[raw_data[is_invalid_date] == True,field] = CANON_NULL_ENCODING
    raw_data = raw_data.drop(columns = is_invalid_date)
    
raw_data.head()

,date_of_birth,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,19560409,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,1804974
febrl3-rec-552-dup-3,19080419,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,6089216
febrl3-rec-988-dup-1,19081128,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,2185997
febrl3-rec-1716-dup-1,19921119,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,4314184
febrl3-rec-1213-org,19991207,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,9144092


## Standardize Dates to common format yyyy/mm/dd

In [45]:
for field in date_fields:
    raw_data[field] = raw_data[field].apply(standardize_date)
    
raw_data.head()

,date_of_birth,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,soc_sec_id
rec_id,,,,,,,,,,
febrl3-rec-1496-org,1956/04/09,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,1804974
febrl3-rec-552-dup-3,1908/04/19,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,6089216
febrl3-rec-988-dup-1,1908/11/28,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,2185997
febrl3-rec-1716-dup-1,1992/11/19,isabelle,,23,gundulu place,currin ga,utakarra,2193,wa,4314184
febrl3-rec-1213-org,1999/12/07,taylor,hathaway,7,yuranigh court,brentwood vlge,,4220,nsw,9144092


## write cleaned output to json file 

In [46]:
# making sure all values are strings before writing to json
assert raw_data.applymap(type).eq(str).all().all(), "not all values are strings!"

assert raw_data.isnull().sum().sum() == 0, "still found a native Python null in the dataset!"

In [47]:
raw_data = raw_data.to_dict(orient = "index")

with open(unlabeled_data_filepath, "w") as json_file:
    json.dump(raw_data, json_file)